In [2]:
import pandas as pd

df = pd.read_csv("inflation_zscores_labeled.csv")

# Show column names clearly
for col in df.columns:
    print(f"'{col}'")

FileNotFoundError: [Errno 2] No such file or directory: 'inflation_zscores_labeled.csv'

In [82]:
df.columns = df.columns.str.strip()

df.rename(columns={
    'CPI MoM %': 'CPI_Z',
    'Core CPI (MoM %)': 'Core_CPI_Z',
    'PPI (MoM %)': 'PPI_Z',
    'Consumer Sentiment Index (UMich)': 'UMich_Infl_Z'
}, inplace=True)


In [84]:
def normalized_contribution(z, weight, cap=2.0):
    # Cap Z-score between -2 and +2
    z = max(min(z, cap), -cap)
    # Normalize to 0–1 scale where -2 → 0, 0 → 0.5, +2 → 1
    scaled = (z + cap) / (2 * cap)
    return scaled * weight


In [86]:
# Headline Inflation
df['Headline_Inflation_SubTheme_Score'] = df['CPI_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

# Core Inflation
df['Core_Inflation_SubTheme_Score'] = df['Core_CPI_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

# Cost Pressures
df['Cost_Pressures_SubTheme_Score'] = df['PPI_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

# Inflation Expectations (from UMich sentiment)
df['Inflation_Expectations_SubTheme_Score'] = df['UMich_Infl_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100


In [88]:
weights_inflation = {
    'Headline_Inflation_SubTheme_Score': 0.3,
    'Core_Inflation_SubTheme_Score': 0.3,
    'Cost_Pressures_SubTheme_Score': 0.2,
    'Inflation_Expectations_SubTheme_Score': 0.2
}

df['Inflation_Theme_Score'] = sum(
    df[col] * weight for col, weight in weights_inflation.items()
).round(2)


In [90]:
print(df[['Date', 'Inflation_Theme_Score']].head())


         Date  Inflation_Theme_Score
0  1985-01-01                  48.52
1  1985-02-01                  73.05
2  1985-03-01                  61.63
3  1985-04-01                  52.96
4  1985-05-01                  57.11


In [100]:
df[['Date',
    'Headline_Inflation_SubTheme_Score',
    'Core_Inflation_SubTheme_Score',
    'Cost_Pressures_SubTheme_Score',
    'Inflation_Expectations_SubTheme_Score',
    'Inflation_Theme_Score',
]].to_csv("inflation_theme_output.csv", index=False)

In [102]:
print(df['UMich_Infl_Z'].describe())
print(df['UMich_Infl_Z'].head(10))


count    4.830000e+02
mean    -3.383537e-16
std      1.001037e+00
min     -4.089246e+00
25%     -6.193839e-01
50%     -1.667933e-01
75%      2.857974e-01
max      3.453932e+00
Name: UMich_Infl_Z, dtype: float64
0   -0.317657
1   -0.015930
2   -0.166793
3    0.285797
4    0.134934
5    0.436661
6   -0.468520
7   -0.468520
8   -0.317657
9    0.285797
Name: UMich_Infl_Z, dtype: float64


In [ ]:
df['Inflation_Expectations_SubTheme_Score'] = df['UMich_Infl_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100
